### 本益比 = 投入成本 / 未來每年收益
### 股票本益比 = 股價 / 公司未來每年收益 = 股價 / 每股盈餘

In [1]:
import requests
import pandas as pd
from io import StringIO
import sqlite3
import datetime
import time
import os
def crawl_price(date):
    
    # 將 date 變成字串 舉例：'20180525' 
    datestr = date.strftime('%Y%m%d')
    datestr_year=date.strftime('%Y')
    datestr_mon=date.strftime('%m')
    datestr_day=date.strftime('%d')

    # 從網站上依照 datestr 將指定日期的股價抓下來
    r = requests.post(f'http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date={ datestr }&type=ALLBUT0999')
    
    # 將抓下來的資料（r.text），其中的等號給刪除
    content = r.text.replace('=', '')
    
    # 將 column 數量小於等於 10 的行數都刪除
    lines = content.split('\n')
    lines = list(filter(lambda l:len(l.split('",')) > 10, lines))
    
    # 將每一行再合成同一行，並用肉眼看不到的換行符號'\n'分開
    content = "\n".join(lines)
    
    # 假如沒下載到，則回傳None（代表抓不到資料）
    if content == '':
        return None
    
    # 將content變成檔案：StringIO，並且用pd.read_csv將表格讀取進來
    df = pd.read_csv(StringIO(content))
    
    # 將表格中的元素都換成字串，並把其中的逗號刪除
    df = df.astype(str)
    df = df.apply(lambda s: s.str.replace(',', ''))
    
    # 將爬取的日期存入 dataframe
    df['date'] = pd.to_datetime(date)
    
    # 將「證券代號」的欄位改名成「stock_id」
    df = df.rename(columns={'證券代號':'stock_id'})
    
    # 將 「stock_id」與「date」設定成index 
    df = df.set_index(['stock_id', 'date'])
    
    # 將所有的表格元素都轉換成數字，error='coerce'的意思是說，假如無法轉成數字，則用 NaN 取代
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))
    
    # 刪除不必要的欄位
    df = df[df.columns[df.isnull().all() == False]]
#     print(df)
        
    if not os.path.isdir(f'C:/Users/User/SideProject/stock_learning/data/{datestr_year}/{datestr_mon}'):
        os.mkdir(f'C:/Users/User/SideProject/stock_learning/data/{datestr_year}/{datestr_mon}')
    
    df.to_csv(f'C:/Users/User/SideProject/stock_learning/data/{datestr_year}/{datestr_mon}/{datestr_day}_price.csv',encoding='utf_8_sig')
    print(datestr)
    return df


#     for i in range(days):
#         crawl_price(datetime.datetime(2021,8,i+1))
#         time.sleep(5)



In [6]:
year = 2012
for j in range(12):    
    for i in range(31): 
        if j+1 == 2:
            if i+1>28:            
                continue
        if j+1 in [4,6,9,11]:
            if i+1 >30:
                continue
                
        crawl_price(datetime.datetime(year,j+1,i+1))
        time.sleep(5)

20120102
20120103
20120104
20120105
20120106
20120109
20120110
20120111
20120112
20120113
20120116
20120117
20120118
20120130
20120131
20120201
20120202
20120203
20120204
20120206
20120207
20120208
20120209
20120210
20120213
20120214
20120215
20120216
20120217
20120220
20120221
20120222
20120223
20120224
20120301
20120302
20120303
20120305
20120306
20120307
20120308
20120309
20120312
20120313
20120314
20120315
20120316
20120319
20120320
20120321
20120322
20120323
20120326
20120327
20120328
20120329
20120330
20120402
20120403
20120405
20120406
20120409
20120410
20120411
20120412
20120413
20120416
20120417
20120418
20120419
20120420
20120423
20120424
20120425
20120426
20120427
20120430
20120502
20120503
20120504
20120507
20120508
20120509
20120510
20120511
20120514
20120515


SSLError: HTTPSConnectionPool(host='www.twse.com.tw', port=443): Max retries exceeded with url: /exchangeReport/MI_INDEX?response=csv&date=20120516&type=ALLBUT0999 (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))

In [50]:
df= pd.read_csv('daily_price.csv',index_col=['stock_id'],)
df= df.loc[~df['收盤價'].isnull()]

conn = sqlite3.connect('test.sqlite3')
df.to_sql('daily_price',conn,if_exists='replace') # to_sql([table名稱],[connection],如果存在怎辦)
df = pd.read_sql('select * from daily_price',conn, index_col=['stock_id'])

,date,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
stock_id,,,,,,,,,,,,,,
1476,2021-08-31,748287,664,413561886,557.0,557.0,543.0,557.0,8.0,555.0,1,557.0,8,26.66
1590,2021-08-31,899855,684,759592932,858.0,860.0,837.0,845.0,8.0,845.0,6,846.0,4,25.12
2207,2021-08-31,491917,449,283667910,575.0,583.0,563.0,583.0,3.0,583.0,36,584.0,2,19.54
2330,2021-08-31,58916407,46791,35867753009,604.0,614.0,598.0,614.0,9.0,613.0,30,614.0,1589,28.73
2379,2021-08-31,1595264,1264,878575746,548.0,556.0,543.0,554.0,6.0,553.0,16,554.0,38,22.65
2454,2021-08-31,7759596,8621,6953519849,917.0,917.0,885.0,901.0,11.0,900.0,1,901.0,35,17.59
3008,2021-08-31,377541,1973,1000410950,2615.0,2680.0,2605.0,2680.0,75.0,2680.0,1,2685.0,9,17.04
3661,2021-08-31,4421942,4776,3062124853,672.0,708.0,669.0,699.0,26.0,698.0,5,699.0,5,39.31
5269,2021-08-31,1076616,1453,2228893615,2000.0,2120.0,1990.0,2075.0,40.0,2075.0,1,2080.0,12,46.49
